### Scrapeando Books to Scrap

In [3]:
# ENTREGABLE - Books to scrape
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

print("¡Bienvenidos al Scrapeador de Books to Scrape! by Rubens")
print("Scrapeando https://books.toscrape.com since 2024")
print("Presione cualquier tecla para scrapear ->")
inicio = input("Presione cualquier tecla para scrapear ->")

# Creamos la lista de libros que sera una lista de diccionarios para los elementos del df
lista_libros = []
# Recorremos todas las paginas
for pagina in range(1,3):
    url = f'https://books.toscrape.com/catalogue/page-{pagina}.html'
    html = req.get(url).content
    soup=bs(html, 'html.parser')

    # Sacamos el listado de libros en cada pagina
    libros = soup.find_all('article')
    
    # Descomentar la siguiente linea para depuracion
    # print(f"\nVoy por la pagina {pagina}\n")
    
    # Recorremos esos libros uno a uno para scrapear todos sus datos    
    for libro in libros:
        try:
            # Diccionario donde almacenamos datos de 1 libro
            dicc_libro = {} 
            
            # Url del catalogo del libro de donde sacamos el resto de los datos
            parte_url_libro = libro.find('h3').find('a').attrs['href']
            url_libro = f'https://books.toscrape.com/catalogue/{parte_url_libro}'
            html_interna = req.get(url_libro).content
            libro_soup=bs(html_interna, 'html.parser')
            
            
            # Cojo la cabecera donde tengo 3 de los elementos que me interesan
            cabecera = libro_soup.find('ul',{'class': 'breadcrumb'}).find_all('li')

            # - Titulo del libro
            titulo_libro = cabecera[3].text.strip()
            dicc_libro['titulo_libro'] = titulo_libro

            # - Categoria del libro
            categoria = cabecera[2].text.strip()
            dicc_libro['categoria'] = categoria

            # - Tipo de Producto
            tipo_producto = cabecera[1].text.strip()
            dicc_libro['tipo_producto'] = tipo_producto
            

            # - Cojo la Portada del libro
            portada = libro_soup.find('img').attrs['src']
            dicc_libro['portada'] = f"<img src='https://books.toscrape.com/{portada.split("..")[2]}'>"
                    
            # - Precio del libro
            precio = libro_soup.find('p', {'class':'price_color'}).text.strip()
            dicc_libro['precio_£'] = float(precio.replace("£", ""))
                
            # - Cojo la Descripcion del libro, si es que hay.
            descripcion = libro_soup.find_all('div', {'id':'product_description'})
            if len(descripcion) > 0:
                dicc_libro['descripcion'] = libro_soup.find_all('p')[3].text.strip()
            else:
                dicc_libro['descripcion'] = "sin descripcion"
                
                
            # Ahora cojo la tabla donde esta el resto de los elementos
            cuerpo = libro_soup.find('table').find_all('tr')
                    
            # - Universal Product Code
            upc = cuerpo[0].find('td').text.strip()
            dicc_libro['upc'] = upc
            
            # - Precio sin impuestos
            precio_sin_impuestos = cuerpo[2].find('td').text.strip()
            dicc_libro['precio_sin_impuestos_£'] = float(precio_sin_impuestos.replace("£", ""))
            
            # - Precio con impuestos
            precio_con_impuestos = cuerpo[3].find('td').text.strip()
            dicc_libro['precio_con_impuestos_£'] = float(precio_con_impuestos.replace("£", ""))
            
            # - Impuestos
            impuestos = cuerpo[4].find('td').text.strip()
            dicc_libro['impuestos_£'] = float(impuestos.replace("£", ""))
                
            # - Sacamos la cadena de Stock   
            stock = cuerpo[5].find('td').text.strip()
            
            if "In stock" in stock:
                # - Tenemos disponibilidad
                dicc_libro['stock'] = 'si'
                # - Cantidad en Stock
                dicc_libro['cantidad'] = int("".join([c for c in stock if c.isdigit()])) # [2,2] -> "".join -> "22"
                # dicc_libro['cantidad2'] = int(stock.replace("In stock (","").replace(" available)",""))
            else:
                # - No tenemos disponibilidad
                dicc_libro['stock'] = 'no'
                dicc_libro['cantidad'] = 0

            # - Cantidad de criticas
            criticas = cuerpo[6].find('td').text.strip()
            dicc_libro['criticas'] = int(criticas)
            
            
            #Appendeamos a nuestra lista de diccionarios de respuesta
            lista_libros.append(dicc_libro) 
        except:
            # Descomentar la siguiente linea para depuracion
            # print(f"{titulo_libro} No se ha podido añadir")
            continue
        else:
            # Descomentar la siguiente linea para depuracion
            # print(f"{titulo_libro} Añadido correctamente")
            continue
    # Comentar la siguiente linea si molesta
    print(f"\nTodos los libros de la pagina {pagina}, añadidos correctamente\nLlevo {len(lista_libros)} registros guardados")

books_df = pd.DataFrame(lista_libros)  
books_df

¡Bienvenidos al Scrapeador de Books to Scrape! by Rubens
Scrapeando https://books.toscrape.com since 2024
Presione cualquier tecla para scrapear ->

Todos los libros de la pagina 1, añadidos correctamente
Llevo 20 registros guardados

Todos los libros de la pagina 2, añadidos correctamente
Llevo 40 registros guardados


,titulo_libro,categoria,tipo_producto,portada,precio_£,descripcion,upc,precio_sin_impuestos_£,precio_con_impuestos_£,impuestos_£,stock,cantidad,criticas
0,A Light in the Attic,Poetry,Books,<img src='https://books.toscrape.com//media/ca...,51.77,It's hard to imagine a world without A Light i...,a897fe39b1053632,51.77,51.77,0.0,si,22,0
1,Tipping the Velvet,Historical Fiction,Books,<img src='https://books.toscrape.com//media/ca...,53.74,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,53.74,53.74,0.0,si,20,0
2,Soumission,Fiction,Books,<img src='https://books.toscrape.com//media/ca...,50.10,"Dans une France assez proche de la nôtre, un h...",6957f44c3847a760,50.10,50.10,0.0,si,20,0
3,Sharp Objects,Mystery,Books,<img src='https://books.toscrape.com//media/ca...,47.82,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,47.82,47.82,0.0,si,20,0
4,Sapiens: A Brief History of Humankind,History,Books,<img src='https://books.toscrape.com//media/ca...,54.23,From a renowned historian comes a groundbreaki...,4165285e1663650f,54.23,54.23,0.0,si,20,0
5,The Requiem Red,Young Adult,Books,<img src='https://books.toscrape.com//media/ca...,22.65,Patient Twenty-nine.A monster roams the halls ...,f77dbf2323deb740,22.65,22.65,0.0,si,19,0
6,The Dirty Little Secrets of Getting Your Dream...,Business,Books,<img src='https://books.toscrape.com//media/ca...,33.34,Drawing on his extensive experience evaluating...,2597b5a345f45e1b,33.34,33.34,0.0,si,19,0
7,The Coming Woman: A Novel Based on the Life of...,Default,Books,<img src='https://books.toscrape.com//media/ca...,17.93,"""If you have a heart, if you have a soul, Kare...",e72a5dfc7e9267b2,17.93,17.93,0.0,si,19,0
8,The Boys in the Boat: Nine Americans and Their...,Default,Books,<img src='https://books.toscrape.com//media/ca...,22.60,For readers of Laura Hillenbrand's Seabiscuit ...,e10e1e165dc8be4a,22.60,22.60,0.0,si,19,0
9,The Black Maria,Poetry,Books,<img src='https://books.toscrape.com//media/ca...,52.15,"Praise for Aracelis Girmay:""[Girmay's] every l...",1dfe412b8ac00530,52.15,52.15,0.0,si,19,0


In [4]:
portada.split("..")[2]

'/media/cache/6d/41/6d418a73cc7d4ecfd75ca11d854041db.jpg'

In [2]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   titulo_libro            20 non-null     object 
 1   categoria               20 non-null     object 
 2   tipo_producto           20 non-null     object 
 3   portada                 20 non-null     object 
 4   £_precio                20 non-null     float64
 5   descripcion             20 non-null     object 
 6   upc                     20 non-null     object 
 7   £_precio_sin_impuestos  20 non-null     float64
 8   £_precio_con_impuestos  20 non-null     float64
 9   £_impuestos             20 non-null     float64
 10  stock                   20 non-null     object 
 11  cantidad                20 non-null     int64  
 12  criticas                20 non-null     object 
dtypes: float64(4), int64(1), object(8)
memory usage: 2.2+ KB


### Pruebas y borradores

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import string as st
import pandas as pd
url = f'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [ ]:
h3_titulos = soup.find_all('ul',{'class': 'breadcrumb'}).find_all('li')[2]

titulos_ofertas = [h3.text.replace('\n','').strip() for h3 in h3_titulos]
titulos_ofertas

In [10]:
h3_titulos = soup.find('ul',{'class': 'breadcrumb'}).find_all('li')[2]
h3_titulos.text.strip()

'Poetry'

In [1]:
stock = "12sdfasdf34"
paco = int("".join([c for c in stock if c.isdigit()]))
paco

1234

In [ ]:
print(soup.prettify())

In [22]:
df[df["descripcion"]=="Sin descripcion"]

,tipo_producto,categoria,titulo_libro,portada,precio,descripcion,upc,precio_sin_impuestos,precio_con_impuestos,impuestos,stock,cantidad,criticas
160,Books,Default,The Bridge to Consciousness: I'm Writing the B...,../../media/cache/39/c7/39c7bebcde5d9643ede71a...,£32.00,Sin descripcion,efc3768127714ec3,£32.00,£32.00,£0.00,si,15,0


In [17]:
print(libro_soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   The Bridge to Consciousness: I'm Writing the Bridge Between Science and Our Old and New Beliefs. | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:30" name="created"/>
  <meta content="
    
" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="../../static/oscar/favicon.